In [ ]:
# ======== COMMON CONFIG (run once) ==========================================
import os, sys, re, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from rolling_framework import Machine   # <-- 프로젝트의 핵심 API

# ---- 파일 경로 --------------------------------------------------------------
DATA_DIR      = "data/"
Y_FILE        = os.path.join(DATA_DIR, "exrets.csv")
SLOPE_FILE    = os.path.join(DATA_DIR, "slope.csv")
YL_FILE       = os.path.join(DATA_DIR, "yl_all.csv")
MACRO_FILE    = os.path.join(DATA_DIR, "MacroFactors.csv")

OUT_DIR       = "./output";  os.makedirs(OUT_DIR, exist_ok=True)

# ---- 샘플/예측 구간 ---------------------------------------------------------
BURN_START, BURN_END     = "197108", "199001"
PERIOD_START, PERIOD_END = "197108", "202312"
HORIZON = 12                           # months ahead

MATURITIES = ["xr_2","xr_3","xr_5","xr_7","xr_10"]

# ---- 유틸 함수 --------------------------------------------------------------
def _load_csv(path, name):
    try:  return pd.read_csv(path, index_col="Time")
    except FileNotFoundError as e:
        sys.exit(f"[ERROR] missing {name} → {e.filename}")

def _align_time(*dfs):
    idx=None
    for d in dfs: idx = d.index if idx is None else idx.intersection(d.index)
    return [d.loc[idx].sort_index() for d in dfs]

def _direct_pairs(slope_cols, y_cols):
    import re
    mk = lambda s: re.search(r"(\d+)", s).group(1) if re.search(r"(\d+)", s) else None
    y_map = {mk(c): c for c in y_cols}
    return [(sc, y_map[mk(sc)]) for sc in slope_cols if mk(sc) in y_map]

def _set_global_seed(seed: int):
    # 외부에서 seed 고정 (가능한 범위)
    try:
        import torch
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)
    except Exception:
        pass
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

# ---- 데이터 로드 & 정렬 ------------------------------------------------------
y     = _load_csv(Y_FILE,   "exrets")
slope = _load_csv(SLOPE_FILE, "slope")
yl    = _load_csv(YL_FILE,   "yl_all")
macro = _load_csv(MACRO_FILE,"MacroFactors")

# 타깃 열 필터
y_cols = [c for c in MATURITIES if c in y.columns]
if not y_cols: sys.exit("[ERROR] MATURITIES not in exrets")
y = y[y_cols]

# 시간축 맞추기
y, slope, yl, macro = _align_time(y, slope, yl, macro)

# slope->y 자동 매핑  ex) slope_2 -> xr_2  (참고: 이번 실험에는 직접 사용 X)
DIRECT_PAIRS = _direct_pairs(slope.columns, y_cols)

print("✓ Loaded data shapes:",
      {k:v.shape for k,v in [("y",y),("slope",slope),("yl",yl),("macro",macro)]})

# =============================================================================
# CASE: Base = slope(OLS), Residual = macro(MLP)  —  10 out of 20 Ensemble
# =============================================================================

# X 구성: slope + macro 모두 포함 (베이스/잔차가 같은 X에서 서로 다른 열을 사용)
X_macro = pd.concat([slope, macro], axis=1)

# ---- ARM 옵션(기본 세팅) ----------------------------------------------------
BASE_OPT = {
    "base_on": True,                          # 베이스 켜기 → CS-Resi 형태
    "base_cols":   list(slope.columns),       # 베이스는 slope 만 사용
    "target_cols": list(y.columns),           # ['xr_2','xr_3','xr_5','xr_7','xr_10']
    "residual_kind": "mlp",                   # 잔차학습은 단일 MLP
    "feature_cols": list(macro.columns),      # 잔차 입력은 macro 만 사용
    "standardize_res": True,                  # 잔차 입력만 표준화
    # MLP 기본 하이퍼(그리드의 기본값)
    "mlp_hidden": (64, 32),
    "mlp_dropout": 0.1,
    "mlp_lr": 1e-3,
    "mlp_wd": 1e-4,        # <-- L2 (weight decay)
    "mlp_epochs": 200,
    "mlp_patience": 20,
    # seed 는 아래 루프에서 주입
}

# ---- Grid (필수 최소) -------------------------------------------------------
GRID = {
    "arm__residual_model__module__hidden": [(64, 32), (128, 64)],
    "arm__residual_model__module__dropout": [0.0, 0.2],
    "arm__residual_model__optimizer__lr": [1e-3, 5e-4],
    "arm__residual_model__optimizer__weight_decay": [0.0, 1e-4],  # L2
}

# ---- 추출 헬퍼: 프레임워크 적응형 -------------------------------------------
def _extract_preds(mach) -> pd.DataFrame:
    """
    Machine 인스턴스에서 OOS 예측 DataFrame을 최대한 호환성 있게 추출.
    (필요 시 프로젝트의 recorder/코어 구조에 맞춰 아래 분기를 추가)
    """
    # 1) 흔한 패턴: recorder가 최종 OOS 예측을 들고있음
    for attr in ["rec", "recorder", "core", "results"]:
        obj = getattr(mach, attr, None)
        if obj is None: 
            continue
        for name in ["oos_pred", "oos_preds", "pred_oos", "pred_test", "pred", "yhat_oos"]:
            if hasattr(obj, name):
                val = getattr(obj, name)
                if isinstance(val, pd.DataFrame):
                    return val
    # 2) 전략 내부 보관
    if hasattr(mach, "strategy") and hasattr(mach.strategy, "oos_pred_"):
        val = mach.strategy.oos_pred_
        if isinstance(val, pd.DataFrame):
            return val
    # 3) 실패 시 에러
    raise RuntimeError("Could not locate OOS predictions in Machine. "
                       "Please expose OOS predictions in recorder/core (e.g., .rec.oos_pred).")

def _extract_metric(mach) -> float:
    """
    Machine 인스턴스에서 OOS MSE 단일 숫자 추출(낮을수록 좋음).
    기본은 Machine.MSEOOS(), 없으면 직접 계산.
    """
    # 1) API가 있으면 그대로 사용
    if hasattr(mach, "MSEOOS"):
        try:
            mse = mach.MSEOOS()
            # Series/DataFrame 인 경우 평균 스칼라화
            if hasattr(mse, "mean"):
                return float(mse.mean())
            return float(mse)
        except Exception:
            pass

    # 2) 직접 계산 (OOS 예측 vs 정답)
    yhat = _extract_preds(mach)
    # 가능한 정답 경로 추정
    y_true = None
    for attr in ["rec", "recorder", "core", "results"]:
        obj = getattr(mach, attr, None)
        if obj is None: 
            continue
        for name in ["oos_true", "y_oos", "y_test", "y_true_oos"]:
            if hasattr(obj, name):
                val = getattr(obj, name)
                if isinstance(val, pd.DataFrame):
                    y_true = val
                    break
        if y_true is not None: break
    if y_true is None:
        # 마지막 수단: 원천 y에서 앙상블 인덱스 구간 추출
        # (recorder가 저장을 안 해 주는 프레임워크일 때)
        y_true = y.loc[yhat.index, yhat.columns]

    diff = y_true.loc[yhat.index, yhat.columns] - yhat
    return float((diff**2).values.mean())

def _r2_score_df(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> pd.Series:
    """열별 R² 계산 (인덱스/열 정렬 후)."""
    y_true = y_true.loc[y_pred.index, y_pred.columns]
    ss_res = ((y_true - y_pred)**2).sum(axis=0)
    ss_tot = ((y_true - y_true.mean(axis=0))**2).sum(axis=0)
    r2 = 1 - ss_res/ss_tot
    return r2

# ---- 20회 실행 → 10개 선발 → 평균 ------------------------------------------
N_RUNS = 20
TOP_K  = 10
SEEDS  = list(range(1, N_RUNS+1))  # [1..20]

runs = []  # [(mse, seed, preds_df)]

print("\n▶ 20 runs start (ARM: base=slope OLS, residual=macro-MLP)")
for seed in SEEDS:
    print(f"  - run seed={seed} ...", flush=True)
    _set_global_seed(seed)

    opt = dict(BASE_OPT)  # 복사
    opt["seed"] = seed    # 내부 모듈에도 seed 주입

    m = Machine(
        X_macro, y, "ARM",
        option=opt, params_grid=GRID,
        burn_in_start=BURN_START, burn_in_end=BURN_END,
        period=[PERIOD_START, PERIOD_END], forecast_horizon=HORIZON
    )
    m.training()

    try:
        preds = _extract_preds(m)  # DataFrame
    except Exception as e:
        # recorder가 OOS 예측을 저장하지 않았다면 전략/머신을 수정해야 함
        raise

    mse = _extract_metric(m)
    runs.append((mse, seed, preds))

# ---- 상위 10개 선발 (MSE 오름차순) ------------------------------------------
runs.sort(key=lambda t: t[0])
selected = runs[:TOP_K]
print(f"\n✓ Selected TOP-{TOP_K} seeds (by OOS-MSE):", [s for _, s, _ in selected])

# ---- 앙상블 평균 ------------------------------------------------------------
# 인덱스/열 정렬 일치화
cols = selected[0][2].columns
idx  = selected[0][2].index
stack = []
for _, seed, pred in selected:
    pred = pred.loc[idx, cols]
    stack.append(pred)
ensemble_pred = sum(stack) / len(stack)

# ---- 앙상블 성능 요약 -------------------------------------------------------
# 정답(y_true) 맞추기: recorder에 없으면 원천 y에서 추출
try:
    # 첫 베스트 run에서 정답 추출 시도 (있다면)
    dummy_m = None
except:
    dummy_m = None

y_true_ens = y.loc[ensemble_pred.index, ensemble_pred.columns]
mse_ens = float(((y_true_ens - ensemble_pred)**2).values.mean())
r2_ens  = _r2_score_df(y_true_ens, ensemble_pred)

print("\n===== ENSEMBLE (10 out of 20) SUMMARY =====")
print(f"OOS-MSE (avg over all targets): {mse_ens: .6f}")
print("R² by target:")
print(r2_ens.to_string())

# ---- 저장 -------------------------------------------------------------------
ens_pred_path = os.path.join(OUT_DIR, "ensemble_pred_10of20.csv")
ens_r2_path   = os.path.join(OUT_DIR, "ensemble_r2_10of20.csv")
ensemble_pred.to_csv(ens_pred_path)
r2_ens.to_frame("R2").to_csv(ens_r2_path)
print(f"\n✓ Saved ensemble predictions  → {ens_pred_path}")
print(f"✓ Saved ensemble R²           → {ens_r2_path}")

DNN_DUAL rolling:   4%|▍         | 20/520 [03:47<1:34:56, 11.39s/it]
